#Import

In [2]:
import time
from pyomo.environ import *
from pyomo.gdp import *
import shutil
import sys
import os.path
from pyomo.gdp import *

In [3]:
%%capture
import sys
import os

import sys, os


if 'google.colab' in sys.modules:
    %pip install idaes-pse --pre >/dev/null 2>/dev/null
    %pip install highspy >/dev/null 2>/dev/null
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

solver_NLO = "ipopt"

assert SolverFactory(solver_NLO).available(), f"Solver {solver_NLO} is not available."

In [4]:
!apt-get install -y glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

#Model

In [6]:
# Create the model
def Process(reformulation=None, solver=None):
    # Create the model
    model = ConcreteModel()

    # Variables
    model.x = Var(range(1, 6), bounds=(0, 10))  # Variables for each process (x1, x2, ..., x5)
    model.y = Var(range(1, 6), bounds=(0, 10))  # Variables for each process (y1, y2, ..., y5)

    # Disjuncts for each process
    def process_rule(disjunct, process):
        model = disjunct.model()
        if process == 1:
            disjunct.cost = Expression(expr=model.x[1]**2 + model.y[1]**2)
            disjunct.constraint = Constraint(expr=model.x[1] + model.y[1] >= 5)
        elif process == 2:
            disjunct.cost = Expression(expr=model.x[2]**3 + model.y[2]**3)
            disjunct.constraint = Constraint(expr=model.x[2] * model.y[2] >= 10)
        elif process == 3:
            disjunct.cost = Expression(expr=model.x[3]**2 + model.y[3]**3)
            disjunct.constraint = Constraint(expr=model.x[3] + model.y[3]**2 >= 8)
        elif process == 4:
            disjunct.cost = Expression(expr=model.x[4]**3 + model.y[4]**2)
            disjunct.constraint = Constraint(expr=model.x[4] * model.y[4] + model.x[4] >= 12)
        elif process == 5:
            disjunct.cost = Expression(expr=model.x[5]**2 + model.y[5]**2 + model.x[5] * model.y[5])
            disjunct.constraint = Constraint(expr=model.x[5] + model.y[5] >= 7)

    # Create disjuncts for each process
    model.process = Disjunct(range(1, 6), rule=process_rule)

    # Disjunction: Choose exactly one process
    model.choose_process = Disjunction(expr=[model.process[i] for i in range(1, 6)])

    # Objective: Minimize the cost of the selected process
    def objective_rule(model):
        return sum(model.process[i].cost for i in range(1, 6))
    model.obj = Objective(rule=objective_rule, sense=minimize)

    if reformulation == None:
        return model
    if solver == None:
        TransformationFactory(reformulation).apply_to(model)

    else:
        TransformationFactory(reformulation).apply_to(model, solver = solver)

    return model

In [7]:
def result(model, end, start):
  print('Time=', end-start)
  print("Objective Value:", model.obj())

  for i in range(1, 6):
      if model.process[i].indicator_var.value == 1:
          print(f"Process {i} is selected.")
          print(f"x{i} = {model.x[i].value}, y{i} = {model.y[i].value}")
          break

#Convex Hull

In [ ]:
# Solve the model
model = Process(reformulation='gdp.hull')
start = time.time()
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='bonmin', tee=True)
end = time.time()

In [ ]:
result(model,end,start)

Time= 5.003036022186279
Objective Value: 63.245552519314316
Process 2 is selected.
x2 = 3.1622776485874873, y2 = 3.1622776485874873


#BigM

In [ ]:
# Solve the model
model = Process('gdp.bigm')
start = time.time()
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='bonmin', tee=True)
end = time.time()

In [ ]:
result(model,end,start)

Time= 2.4269444942474365
Objective Value: 12.499999950065577
Process 1 is selected.
x1 = 2.499999995001, y1 = 2.499999995001


#Logic-based Outer Approximation (LOA)

In [17]:
# Solve the model
model = Process()
start = time.time()
a= SolverFactory
results = SolverFactory('gdpopt.loa').solve(model, mip_solver='glpk', nlp_solver = 'ipopt')
end = time.time()

In [18]:
result(model,end,start)

Time= 0.3022592067718506
Objective Value: 12.49999976086124
Process 1 is selected.
x1 = 2.4999999752505904, y1 = 2.4999999752505904


#Global Logic-based Outer Approximation (GLOA)

In [ ]:
# Solve the model
model = Process()
start = time.time()
a= SolverFactory
results = SolverFactory('gdpopt.gloa').solve(model)
end = time.time()

In [ ]:
result(model,end,start)

#Relaxation with Integer Cuts (RIC)

In [21]:
# Solve the model
model = Process()
start = time.time()
a= SolverFactory
results = SolverFactory('gdpopt.ric').solve(model, mip_solver='glpk')
end = time.time()

In [22]:
result(model,end,start)

Time= 0.26461005210876465
Objective Value: 12.499999760859243
Process 1 is selected.
x1 = 2.4999999752505904, y1 = 2.4999999752505904


#Logic-based Branch-and-Bound (LBB)

In [27]:
# Solve the model
model = Process()
start = time.time()
a= SolverFactory
results = SolverFactory('gdpopt.lbb').solve(model, minlp_solver='ipopt')
end = time.time()

In [28]:
result(model,end,start)

Time= 0.6028823852539062
Objective Value: 12.499999760859309
Process 1 is selected.
x1 = 2.4999999752505904, y1 = 2.4999999752505904
